In [ ]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [ ]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## חקירה מהירה וגרפים זריזים (Exploration & Quick Plots)

בשלבי חקירה ראשוניים (EDA) של נתונים ניסיוניים בפיזיקה, נרצה לקבל תחושת בטן מהירה על ההתפלגויות, הגדלים הבולטים, וקשרים בין משתנים.  
ב־Pandas וכלים נלווים נשתמש ב־`value_counts` לספירת קטגוריות (למשל סוג חיישן/פאנל), `nlargest` למציאת תצפיות או קטגוריות בולטות, ובמטריצת **מתאמים** (correlations) לזיהוי קשרים ליניאריים (למשל בין **מתח**, **זרם**, ו־**עוצמת קרינה**).

#### value_counts ו־nlargest
- `value_counts()` עוזר לראות אילו קטגוריות דומיננטיות (סוג ניסוי/פאנל/חומר).
- `nlargest(k, col)` מאתר את ה־k הגדולים ביותר בעמודה — שימושי לזיהוי שבועות עם תפוקה מקסימלית או חיישנים חריגים.

#### מתאמים (correlations)
מטריצת מתאמים (למשל `df.corr(numeric_only=True)`) מסייעת להבין קשרים בין משתנים נומריים —  
במערכת פוטו־וולטאית, מתאם חיובי בין **קרינה סולארית** ל־**הספק** הוא צפוי; מתאם עם **טמפרטורה** יכול להיות מורכב יותר.

#### סיור מהיר בגרפים
כאשר `matplotlib` זמינה, נצייר קווים/עמודות להמחשה מהירה:  
- זמן ← תפוקה שבועית (energy/power).  
- ברים קטגוריאליים ל־Top-N.  
- הדגשה צבעונית לטופ־3 שבועות.

### תרגילים: חקירה וגרפים לנתונים פיסיקליים
נשתמש בדאטה מדומה של מערכת סולארית: קריאות יומיות (kWh), יחד עם מאפייני ניסוי (סוג פאנל) ופרמטרים משלימים (קרינה, טמפרטורה).

#### תרגיל 1: ניתוח והדמיה של תפוקת אנרגיה שבועית

בתרגיל זה נחקור את תפוקת האנרגיה של מערכת סולארית לאורך זמן.  
נניח שיש לנו נתונים יומיים על **אנרגיה שהופקה (daily_energy_kWh)**, יחד עם פרמטרים סביבתיים כמו **קרינה סולארית (irradiance)** ו־**טמפרטורה (temperature)**.  

נרצה:
1. **לאגד את הנתונים לפי שבוע** כדי לראות מגמות רחבות (באמצעות `resample('W').sum()`),  
2. **לאתר את שלושת השבועות בעלי התפוקה הגבוהה ביותר** באמצעות `nlargest(3)`,  
3. ואם `matplotlib` זמין — **להציג גרף קווי** של סך האנרגיה השבועית,  
   עם הדגשה של שלושת השבועות המובילים (Top-3) על־גבי הגרף.  

תרגיל זה מדגים כיצד ניתן לגלות תקופות שיא במדידות פיזיקליות — לדוגמה, שבועות של קרינה גבוהה במיוחד או יעילות מרבית של הפאנלים.



In [ ]:
# --- Exercise 1: Plot weekly energy and highlight top 3 weeks ---
import pandas as pd
import numpy as np

# Optional plotting (guard if matplotlib is unavailable)
try:
    import matplotlib.pyplot as plt
    MATPLOTLIB_OK = True
except Exception:
    MATPLOTLIB_OK = False

# Create synthetic daily production and features
np.random.seed(7)
dates = pd.date_range("2025-01-01", periods=120, freq="D")
irradiance = np.clip(np.random.normal(650, 120, size=len(dates)), 250, 900)  # W/m^2
temperature = np.random.normal(25, 5, size=len(dates))                      # °C

# Simulate panel categories
panel_types = np.random.choice(["Mono", "Poly", "ThinFilm"], size=len(dates), p=[0.5, 0.35, 0.15])

# Daily energy (kWh) ~ irradiance with noise; mild temp penalty
energy = (irradiance * 0.22) - (np.maximum(0, temperature - 28) * 2.5) + np.random.normal(0, 15, size=len(dates))
energy = np.clip(energy, 20, None)  # No negative production

df = pd.DataFrame({
    "timestamp": dates,
    "panel_type": panel_types,
    "irradiance_Wm2": irradiance,
    "temperature_C": temperature,
    "daily_energy_kWh": energy
}).set_index("timestamp")

# Weekly aggregation (sum of energy per week)
weekly = df["daily_energy_kWh"].resample("W").sum().to_frame(name="weekly_energy_kWh")

# Find Top-3 weeks
top3 = weekly["weekly_energy_kWh"].nlargest(3)

print("Weekly energy (kWh) - head:")
display(weekly.head())

print("\nTop-3 weeks by total energy:")
display(top3)

# Plot if possible
if MATPLOTLIB_OK:
    # --- Plot weekly energy and highlight top 3 weeks ---
    ax = weekly["weekly_energy_kWh"].plot(kind="line", marker="o", title="Weekly Solar Energy (kWh)")
    # Highlight top-3 with scatter
    ax.scatter(top3.index, top3.values, s=80, zorder=3)
    for ts, val in top3.items():
        ax.annotate(f"Top\n{val:.0f}", (ts, val), textcoords="offset points", xytext=(0, -25), ha="center")
    ax.set_xlabel("Week")
    ax.set_ylabel("Energy (kWh)")
    plt.show()
else:
    print("\n[Info] matplotlib not available - showing table only.")


#### תרגיל 2: מתאמים בין משתנים ופילוח קטגוריאלי

1) חשב מטריצת מתאמים בין **irradiance_Wm2**, **temperature_C**, ו־**daily_energy_kWh**.  
2) השתמש ב־`value_counts()` לראות את התפלגות סוגי הפאנלים.  
3) חשב ממוצע **daily_energy_kWh** לפי `panel_type`, הצג את **שלושת** הסוגים בעלי הממוצע הגבוה ביותר (אם יש יותר משלושה, בחר Top-3).  
4) אם `matplotlib` זמין, הצג גרף עמודות ל־Top-3 קטגוריות.


In [ ]:
# --- Exercise 2: Correlations and top categories ---
# 1) Correlation matrix
corr = df[["irradiance_Wm2", "temperature_C", "daily_energy_kWh"]].corr(numeric_only=True)
print("Correlation matrix:")
display(corr)

# 2) Category distribution
print("\nPanel type distribution (value_counts):")
display(df["panel_type"].value_counts())

# 3) Mean energy by panel type and top-3
mean_by_type = df.groupby("panel_type")["daily_energy_kWh"].mean().sort_values(ascending=False)
top3_types = mean_by_type.nlargest(3)

print("\nMean daily energy by panel type:")
display(mean_by_type)

print("\nTop-3 panel types by mean daily energy:")
display(top3_types)

# 4) Optional bar plot
if 'MATPLOTLIB_OK' in globals() and MATPLOTLIB_OK:
    ax = top3_types.plot(kind="bar", title="Top-3 Panel Types by Mean Daily Energy")
    ax.set_xlabel("Panel Type")
    ax.set_ylabel("Mean Daily Energy (kWh)")
    for i, v in enumerate(top3_types.values):
        ax.text(i, v, f"{v:.1f}", ha="center", va="bottom")
    plt.show()


#### תרגיל 3 : Weekly “Revenue” מדומה והצלבת Top-3
לפעמים יש קשר בין תפוקה לבין הכנסה (למשל מכירת חשמל לרשת).  
ניצור **הכנסה שבועית מדומה** ע״י הכפלת האנרגיה במחיר־ליחידה משתנה, נצייר אותה, ונשווה **Top-3** שבועות באנרגיה מול הכנסה.

In [ ]:
# --- Exercise 3 (Optional): Weekly revenue proxy & cross-compare top-3 ---
# Create a synthetic weekly price per kWh (random within a band)
weekly_price = pd.Series(
    np.random.uniform(0.45, 0.65, size=len(weekly)),
    index=weekly.index,
    name="price_per_kWh"
)
weekly_with_rev = weekly.join(weekly_price)
weekly_with_rev["weekly_revenue"] = weekly_with_rev["weekly_energy_kWh"] * weekly_with_rev["price_per_kWh"]

print("Weekly revenue (synthetic):")
display(weekly_with_rev.head())

# Identify top-3 by revenue
top3_rev = weekly_with_rev["weekly_revenue"].nlargest(3)
print("\nTop-3 weeks by revenue:")
display(top3_rev)

# Compare overlap with top-3 energy weeks
overlap_weeks = set(top3.index.date).intersection(set(top3_rev.index.date))
print(f"\nOverlap between Top-3 energy and Top-3 revenue weeks: {len(overlap_weeks)}")
print("Overlapping week dates:", sorted(overlap_weeks))

if 'MATPLOTLIB_OK' in globals() and MATPLOTLIB_OK:
    ax = weekly_with_rev["weekly_revenue"].plot(marker="o", title="Weekly Revenue (synthetic)")
    ax.scatter(top3_rev.index, top3_rev.values, s=80, zorder=3)
    for ts, val in top3_rev.items():
        ax.annotate(f"Top\n{val:.0f}", (ts, val), textcoords="offset points", xytext=(0, -25), ha="center")
    ax.set_xlabel("Week")
    ax.set_ylabel("Revenue")
    plt.show()


`````{admonition} סיכום
:class: tip
- קיבלנו תמונה מהירה של הנתונים באמצעות `value_counts`, `nlargest`, ומטריצת מתאמים.  
- זיהינו **Top-3** שבועות בתפוקה והדגשנו אותם בגרף.  
- בחנו קשרים בין משתנים פיזיקליים (קרינה/טמפרטורה/תפוקה), ופילחנו לפי קטגוריות (סוג פאנל).  
- הרחבנו לאנליזה נגזרת (Revenue) כדי להדגים שימוש זהה בכלי החקירה.

כלים אלו מספקים **תגובה מהירה** לשאלות ראשוניות ומכוונים אותנו להמשך חקירה מעמיקה או לבניית מודלים.
`````


In [1]:
import json
from jupyterquiz import display_quiz

quiz_json = \
'''
[{
  "question": "בעת ניתוח נתוני מערכת סולארית, נרצה לזהות את שלושת השבועות שבהם ייצור האנרגיה הכולל (weekly_energy_kWh) היה הגבוה ביותר, ולהציגם בגרף קווי.<br><br>איזה רצף פעולות מתאים לכך בפנדס?",
  "type": "many_choice",
  "answers": [
    {
      "answer": "להשתמש ב־resample('W').sum() כדי לאחד מדידות לפי שבוע, ולאחר מכן ב־nlargest(3) למציאת השבועות בעלי הערכים הגבוהים ביותר, ולצייר גרף קווי עם סימון השבועות הללו.",
      "correct": true,
      "feedback": "נכון! זהו השילוב הנכון: דגימה שבועית בעזרת resample, סינון Top-3 באמצעות nlargest, והצגה ויזואלית עם plot."
    },
    {
      "answer": "להשתמש ב־groupby('week').mean() ולמיין את כל השבועות בסדר עולה לפי האנרגיה.",
      "correct": false,
      "feedback": "לא מדויק — groupby דורש עמודת שבוע מוגדרת מראש, בעוד ש־resample מטפל ישירות באינדקס הזמן."
    },
    {
      "answer": "להשתמש ב־rolling(7).sum() כדי לחשב סכום נע, ולאחר מכן ב־sort_values() כדי לזהות את השבועות החזקים ביותר.",
      "correct": false,
      "feedback": "לא — rolling מחשב ממוצע נע ואינו מבצע דגימה שבועית אמיתית כמו resample."
    },
    {
      "answer": "להשתמש ב־pivot_table ולחשב סכום לפי קטגוריה של שבוע ולא לפי תאריך.",
      "correct": false,
      "feedback": "לא נכון — pivot_table מתאים לנתונים קטגוריאליים, לא לדגימה לפי אינדקס זמן."
    }
  ]
}]
'''

myquiz = json.loads(quiz_json)
display_quiz(myquiz)


<IPython.core.display.Javascript object>